In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
data = pd.read_csv('rawdata/2014.txt', sep = ';')

/Users/adamlevin/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (20,21,22,27,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.columns = np.array([str(i).strip() for i in list(data.columns.values)]) #remove space in col names

In [4]:
data.columns

Index([u'Tour Code', u'Tour Description', u'Year', u'Tourn.#', u'Player #',
       u'Course #', u'Permanent Tournament #', u'Player First Name',
       u'Player Last Name', u'Round', u'Tournament Name', u'Course Name',
       u'Hole', u'Hole Score', u'Par Value', u'Yardage', u'Shot',
       u'Shot Type(S/P/D)', u'# of Strokes', u'From Location(Scorer)',
       u'From Location(Enhanced)', u'To Location(Laser)',
       u'To Location(Enhanced)', u'Distance', u'Distance to Pin',
       u'In the Hole Flag', u'Around the Green Flag', u'1st Putt Flag',
       u'Distance to Hole after the Shot', u'Time', u'Lie', u'Elevation',
       u'Slope', u'X Coordinate', u'Y Coordinate', u'Z Coordinate',
       u'Distance from Center', u'Distance from Edge', u'Date'],
      dtype='object')

In [5]:
data['X Coordinate'] = [str(i).replace(' ','') for i in data['X Coordinate']] #remove space in coordinates cols
data['Y Coordinate'] = [str(i).replace(' ','') for i in data['Y Coordinate']]
data['Z Coordinate'] = [str(i).replace(' ','') for i in data['Z Coordinate']]

In [6]:
data['X Coordinate'] = [str(i).replace(',','') for i in data['X Coordinate']] #remove commas in coordinates cols
data['Y Coordinate'] = [str(i).replace(',','') for i in data['Y Coordinate']]
data['Z Coordinate'] = [str(i).replace(',','') for i in data['Z Coordinate']]

In [7]:
data['X Coordinate'] = ['-' + str(i)[:len(str(i))-1] if str(i)[len(str(i))-1]=='-' else i for i in data['X Coordinate']] #putting negative in front
data['Y Coordinate'] = ['-' + str(i)[:len(str(i))-1] if str(i)[len(str(i))-1]=='-' else i for i in data['Y Coordinate']]
data['Z Coordinate'] = ['-' + str(i)[:len(str(i))-1] if str(i)[len(str(i))-1]=='-' else i for i in data['Z Coordinate']]

In [8]:
data['X Coordinate'] = pd.to_numeric(data['X Coordinate'])
data['Y Coordinate'] = pd.to_numeric(data['Y Coordinate'])
data['Z Coordinate'] = pd.to_numeric(data['Z Coordinate'])

In [9]:
#unique Course-Round-Hole Tuples
uCRHtps = list(itertools.product(np.unique(data['Course Name']),np.unique(data['Round']),np.unique(data['Hole'])))

In [10]:
from scipy.optimize import fmin_tnc

In [11]:
# coordinates of hole are not given. must be imputed.
# does the distance use the x,y, and z coordinates or just the x and y coordinates?
# test: first find the hole using the x,y, and z coordinates and record the average difference between calculated hole
# location and ball and recorded distance. then do the same using just the x and y coodinates. see which is better.

In [12]:
# def f (a):
#     x0,y0,z0 = a[0],a[1],a[2]
#     return sum((((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5-d)**2)/len(x)

# def find_the_hole ():
#     xopt = fmin_tnc(f,[x0,y0,z0],approx_grad=1)[0].tolist()
#     return f(xopt)

In [13]:
# #finding the coordinates of the hole
# aveds=[]
# for u,i in enumerate(uCRHtps):
#     if (u+1)%100==0:
# #         print u
#         print u+1,sum(aveds)/len(aveds)
#     subset = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2])) \
#                   & (data['Distance to Hole after the Shot']!=0) & (data['X Coordinate']!=0) & data['Y Coordinate']!=0]
#     if subset.shape[0] == 0:
#         continue
#     d = subset['Distance to Hole after the Shot'].values/12.0
#     x = subset['X Coordinate'].values
#     y = subset['Y Coordinate'].values
#     z = subset['Z Coordinate'].values
#     sorted_subset = subset.sort_values('Distance to Hole after the Shot')
#     x0 = sorted_subset['X Coordinate'].values[0] ##assume that closest ball recorded to hole does not have an error
#     y0 = sorted_subset['Y Coordinate'].values[0]
#     z0 = sorted_subset['Z Coordinate'].values[0]
#     subset.insert(len(subset.columns),'approx_dist',((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5)
#     # dist_diff is difference between recorded distance and distance approximated from location of closest recorded shot
#     subset.insert(len(subset.columns),'dist_diff', \
#                   np.absolute(subset['approx_dist'].values - subset['Distance to Hole after the Shot'].values/12.0))
#     before = subset.shape[0]
#     # remove very inconsistant shots from record which are likely mistakes
#     subset = subset[subset['dist_diff']<sorted_subset['Distance to Hole after the Shot'].values[0]]
#     after = subset.shape[0]
#     if before-after>0:
#         print u, before-after
#     d = subset['Distance to Hole after the Shot'].values/12.0
#     x = subset['X Coordinate'].values
#     y = subset['Y Coordinate'].values
#     z = subset['Z Coordinate'].values
#     a = find_the_hole()
#     aveds.append(a)
# print sum(aveds)/len(aveds)

In [14]:
# def f (a):
#     x0,y0 = a[0],a[1]
#     return sum((((x-x0)**2 + (y-y0)**2)**.5-d)**2)/len(x)

# def find_the_hole ():
#     xopt = fmin_tnc(f,[x0,y0],approx_grad=1)[0].tolist()
#     return f(xopt)

In [15]:
# aveds=[]
# for u,i in enumerate(uCRHtps):
#     if (u+1)%100==0:
# #         print u
#         print u+1,sum(aveds)/len(aveds)
#     subset = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2])) \
#                   & (data['Distance to Hole after the Shot']!=0) & (data['X Coordinate']!=0) & data['Y Coordinate']!=0]
#     if subset.shape[0] == 0:
#         continue
#     d = subset['Distance to Hole after the Shot'].values/12.0
#     x = subset['X Coordinate'].values
#     y = subset['Y Coordinate'].values
#     sorted_subset = subset.sort_values('Distance to Hole after the Shot')
#     x0 = sorted_subset['X Coordinate'].values[0] ##assume that closest ball recorded to hole does not have an error
#     y0 = sorted_subset['Y Coordinate'].values[0]
#     subset.insert(len(subset.columns),'approx_dist',((x-x0)**2 + (y-y0)**2)**.5)
#     # dist_diff is difference between recorded distance and distance approximated from location of closest recorded shot
#     subset.insert(len(subset.columns),'dist_diff', \
#                   np.absolute(subset['approx_dist'].values - subset['Distance to Hole after the Shot'].values/12.0))
#     before = subset.shape[0]
#     # remove very inconsistant shots from record which are likely mistakes
#     subset = subset[subset['dist_diff']<sorted_subset['Distance to Hole after the Shot'].values[0]]
#     after = subset.shape[0]
#     if before-after>0:
#         print u, before-after
#     d = subset['Distance to Hole after the Shot'].values/12.0
#     x = subset['X Coordinate'].values
#     y = subset['Y Coordinate'].values
#     a = find_the_hole()
#     aveds.append(a)
# print sum(aveds)/len(aveds)

In [47]:
# based on results of number of points that have implausible coordinates, and average error between the recorded dist.
# and this dist. calculated with the imputed coordinates of the hole, it is clear that the recorded distances use 
# the x,y, and z coordinates.
# now recreate dataframe with concatonation of all subsets - without bunk records and with the x,y, and z coordinates
# of the hole.

In [16]:
def f (a):
    x0,y0,z0 = a[0],a[1],a[2]
    return sum((((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5-d)**2)/len(x)

def find_the_hole ():
    xopt = fmin_tnc(f,[x0,y0,z0],approx_grad=1)[0].tolist()
    return xopt

In [60]:
# initializing new data frame with two rows which will be deleted after
newdata = pd.DataFrame(data.loc[1:2,:])

#finding the coordinates of the hole and recording the results
#must treat shots that were holed out differently since the coordinates are recorded as 0 for those shots
for u,i in enumerate(uCRHtps):
    if (u+1)%100==0:
        print u
#         print newdata.shape
    subset = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2])) \
                  & (~((data['Distance to Hole after the Shot']!=0) & \
                       ((data['X Coordinate']==0) | (data['Y Coordinate']==0) | (data['Z Coordinate']==0))))]
    if subset[subset['Distance to Hole after the Shot']!=0].shape[0] == 0:
        continue
    start = subset.shape[0]
    subset = subset.sort_values('Distance to Hole after the Shot')
    d = subset[subset['Distance to Hole after the Shot']!=0]['Distance to Hole after the Shot'].values/12.0
    x = subset[subset['Distance to Hole after the Shot']!=0]['X Coordinate'].values
    y = subset[subset['Distance to Hole after the Shot']!=0]['Y Coordinate'].values
    z = subset[subset['Distance to Hole after the Shot']!=0]['Z Coordinate'].values
    x0 = subset[subset['Distance to Hole after the Shot']!=0]['X Coordinate'].values[0] ##assume that closest ball recorded to hole does not have an error
    y0 = subset[subset['Distance to Hole after the Shot']!=0]['Y Coordinate'].values[0]
    z0 = subset[subset['Distance to Hole after the Shot']!=0]['Z Coordinate'].values[0]
    subset.insert(len(subset.columns),'approx_dist',np.array([0]*subset[subset['Distance to Hole after the Shot']==0].shape[0] \
                  + (((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5).tolist()))
    # dist_diff is difference between recorded distance and distance approximated from location of closest recorded shot
    subset.insert(len(subset.columns),'dist_diff', \
                  np.absolute(subset['approx_dist'].values - subset['Distance to Hole after the Shot'].values/12.0))
    # remove very inconsistant shots from record which are likely mistakes
    subset = subset[subset['dist_diff']<subset[subset['Distance to Hole after the Shot']!=0]['Distance to Hole after the Shot'].values[0]+1]
    finish = subset.shape[0]
    d = subset[subset['Distance to Hole after the Shot']!=0]['Distance to Hole after the Shot'].values/12.0
    x = subset[subset['Distance to Hole after the Shot']!=0]['X Coordinate'].values
    y = subset[subset['Distance to Hole after the Shot']!=0]['Y Coordinate'].values
    z = subset[subset['Distance to Hole after the Shot']!=0]['Z Coordinate'].values
    a = find_the_hole()
    subset.drop('approx_dist',axis=1,inplace=True)
    subset.drop('dist_diff',axis=1,inplace=True)
    subset.insert(len(subset.columns),'Hole X Coordinate',np.array([a[0]]*subset.shape[0]))
    subset.insert(len(subset.columns),'Hole Y Coordinate',np.array([a[1]]*subset.shape[0]))
    subset.insert(len(subset.columns),'Hole Z Coordinate',np.array([a[2]]*subset.shape[0]))
    subset.insert(len(subset.columns),'dist_w_impute',np.array([0]*subset[subset['Distance to Hole after the Shot']==0].shape[0] \
                  + (((x-a[0])**2 + (y-a[1])**2 + (z-a[2])**2)**.5).tolist()))
    subset.insert(len(subset.columns),'dist_diff', \
                  np.absolute(subset['dist_w_impute'].values - subset['Distance to Hole after the Shot'].values/12.0))
    mean_err = subset[subset['Distance to Hole after the Shot']!=0]['dist_diff'].mean()
    if mean_err>.3:
        print u, mean_err
    if start-finish>5:
        print u, start-finish
    newdata = newdata.append(subset)

newdata.drop(newdata.head(2).index, inplace=True)
print newdata.shape

99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599
1699
1799
1899
1999
2099
2199
2299
2399
2499
2599
2699
2799
2899
2999
(985714, 44)


In [45]:
# pd.options.display.max_rows = 2421
# pd.reset_option("^display")


In [58]:
print data.shape
print newdata.shape

(1106109, 39)
(985714, 44)


In [61]:
newdata.to_csv('2014_with_hole_coordinates.csv')